In [11]:
import csv
import re

In [68]:
#Set paths for different files
coreorder1path = "Collected Fall 2017/results-survey955753_CAL P3 Core Order1 - English - V2.2.csv"
coreorder2path = "Collected Fall 2017/results-survey727772_CAL P3 Core Order2 - English - V2.2.csv"
coreorder3path = "Collected Fall 2017/results-survey275281_CAL P3 Core Order3 - English - V2.2.csv"
coreorder4path = "Collected Fall 2017/results-survey152661_CAL P3 Core Order4 - English - V2.2.csv"
supporder1path = "Collected Fall 2017/results-survey282158_CAL P3 Order1 SUPP - English - V2.2.csv"
supporder2path = "Collected Fall 2017/results-survey885882_CAL P3 Order 2 SUPP - English - V2.2.csv"
supporder3path = "Collected Fall 2017/results-survey175111_CAL P3 Order3 SUPP - English - V2.2.csv"
supporder4path = "Collected Fall 2017/results-survey411556_CAL P3 Order4 SUPP - English - V2.2.csv"
corefilepaths = [coreorder1path,coreorder2path,coreorder3path,coreorder4path]
suppfilepaths = [supporder1path,supporder2path,supporder3path,supporder4path]

In [69]:
def processCSVFile(filepath):
    with open(filepath) as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
        # process header rom
        header = spamreader.next()
        sentences = []
        clips = []
        answerTypes = []
        groups = []
        answersets = []
        partIds = []
        i=8
        for q in header[8:-1]:
            if "[" not in q:
                cutoff = i
                break
            clip = q.split("[")[0]
            sentence = q.split("[")[2][:-1]
            if "False" in q:
                answertype = "Rating"
            else:
                answertype = "Alt"
            if clip[-1] == "B":
                group = "B"
            elif clip[0] == "T":
                group = "TRAINING"
            else:
                group = "A"
            sentences.append(sentence)
            clips.append(clip)
            answerTypes.append(answertype)
            groups.append(group)
            i+=1
        #print i
        #Extract answers
        for row in spamreader:
            if len(row)==0:
                break
            partIds.append(row[7])
            answers = row[8:cutoff]
            answersets.append(answers)
    return sentences,clips,answerTypes,groups,answersets,partIds

In [70]:
def CreateDataframe(filepathlist,outputfilename):
    dataframe = []
    dataframe.append(("PartID","Order","Clip","Group","Sentence","AnswerType","Answer"))
    for i in range (1,5):
        filepath = filepathlist[i-1]
        order = "Order"+str(i)
        sentences,clips,answerTypes,groups,answersets,partIds = processCSVFile(filepath)
        for i in range (0,len(partIds)):
            for j in range (0,len(clips)):
                dataframe.append((partIds[i],order,clips[j],groups[j],sentences[j],answerTypes[j],answersets[i][j]))
    with open(outputfilename, 'wb') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerows(dataframe)
                
                #print len(sentences)
#print len(clips)
#print len(answerTypes)
#print len(answersets[0])


#print dataframe

In [71]:
#Generate core dataframe
CreateDataframe(corefilepaths,"CoreData.csv")
#Generate supp dataframe
CreateDataframe(suppfilepaths,"SuppData.csv")
